In [1]:
import os
import numpy as np
import plotly.graph_objs as go

In [2]:
traceCats = ['office-wifi', 'rest-wifi']
ccaList = ['0', '4', '5', '6', '7', '8', '9']
ccaMap = ["Gcc", "Nada", "", "Fixed", 
  "Cubic", "Bbr", "Copa", "Yeah", 
  "Illinois", "Vegas", "Htcp", "Bic", 
  "LinuxReno", "Scalable", "", ""]
queueSize = 250

In [4]:
for cca in ccaList:
    os.system ("zsh app2framedelay.sh ns-3.34/logs-design-cca/%s_Video1" % cca)

In [5]:
bpsAll, rttAll, fdAll, qutilAll = {}, {}, {}, {}
for cca in ccaList:
    bpsAll[cca], rttAll[cca], fdAll[cca], qutilAll[cca] = {}, {}, {}, {}
    for traceCat in traceCats:
        bpsAll[cca][traceCat] = []
        rttAll[cca][traceCat] = []
        fdAll[cca][traceCat] = []
        qutilAll[cca][traceCat] = []
    dirname = "ns-3.34/logs-design-cca/%s_Video1/PfifoFast_0_%d" % (cca, queueSize)
    for trace in os.listdir (dirname):
        isInterest = False
        for traceCat in traceCats:
            if traceCat in trace:
                isInterest = True
                break
        if not isInterest:
            continue
        bpsTrace, rttTrace, fdTrace, qutilTrace = [], [], [], []
        with open (os.path.join (dirname, trace, 'bw.tr')) as f:
            for line in f.readlines ():
                bpsTrace.append (int (line.split ()[4]))
                rttTrace.append (int (line.split ()[6]))
        with open (os.path.join (dirname, trace, 'framedelay.tr')) as f:
            for line in f.readlines ():
                fdTrace.append (int (line.split ()[1]))
        with open (os.path.join (dirname, trace, 'qdisc.tr')) as f:
            for line in f.readlines ():
                qutilTrace.append (float (line.split ()[8]))
        
        bwTrace = []
        with open (os.path.join ("ns-3.34/traces", trace)) as f:
            for line in f.readlines ():
                bwTrace.append (float (line.split ('M')[0]))
        
        for traceCat in traceCats:
            if traceCat in trace:
                bpsAll[cca][traceCat].append (np.mean (bpsTrace) / np.mean (bwTrace))
                rttAll[cca][traceCat].append (np.mean (rttTrace)) # np.percentile (rttTrace, 95))
                fdAll[cca][traceCat].append (np.percentile (fdTrace, 95))
                qutilAll[cca][traceCat].append (np.percentile (qutilTrace, 95))
                break

In [10]:
plotData = []
for cca in ccaList:
    bpsReduced, rttReduced, fdReduced, qutilReduced = [], [], [], []
    for traceCat in traceCats:
        bpsReduced.append (np.mean (bpsAll[cca][traceCat]))
        rttReduced.append (np.mean (rttAll[cca][traceCat]))
        fdReduced.append (np.mean (fdAll[cca][traceCat]))
        qutilReduced.append (np.mean (qutilAll[cca][traceCat]))
    plotData.append (go.Scatter (x=[np.mean (fdReduced)], y=[np.mean (qutilReduced) / queueSize], name=ccaMap[int (cca, 16)], mode='markers'))
    print ("%.3f %.3f %.3f %.3f %s" % (
        np.mean (rttReduced), np.mean (qutilReduced) / queueSize, 
        np.mean (fdReduced), np.mean (qutilReduced) / queueSize, 
        ccaMap[int (cca, 16)]))
fig = go.Figure (data=plotData)
fig.update_layout (title='CCA', xaxis_title='Average RTT (ms)', yaxis_title='Queue utilization')
fig.show ()

52.122 0.101 113.347 0.101 Gcc
193.772 0.938 1013.635 0.938 Cubic
89.102 0.520 357.319 0.520 Bbr
52.716 0.066 163.742 0.066 Copa
151.701 0.859 543.054 0.859 Yeah
193.795 0.954 874.408 0.954 Illinois
61.040 0.136 194.203 0.136 Vegas
